In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing

#model building
import torch
import torchvision
from torchvision.transforms import v2
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from time import sleep


#plotting and evalueation
import seaborn as sns


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [21]:
#reading the data
kenya_train_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Kenya_training.csv")
kenya_test_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Kenya_training.csv")

spain_train_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Spain_training.csv")
spain_test_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/Spain_validation.csv")

vietnam_train_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/VNM_training.csv")
vietnam_test_df = pd.read_csv("geoai-challenge-for-agricultural-plastic-cover-mapping-with-satellite-imagery20240708-24674-1c1nnx3/VNM_testing.csv")

spain_test_df.head(5)
# vietnam_train_df.head(5)

,ID,lon,lat,blue_p50,green_p50,nir_p50,nira_p50,re1_p50,re2_p50,re3_p50,red_p50,swir1_p50,swir2_p50,VV_p50,VH_p50
0,1,-3.571567,36.797285,5062.0,5162.0,6086.0,5923.0,5357.5,5764.0,5942.5,5202.0,4525.0,3652.5,-6.853568,-11.053417
1,2,-3.195622,36.785337,2829.0,3320.0,4007.0,4399.5,4393.0,4366.0,4389.0,3687.0,4112.0,3281.0,-8.282629,-15.775291
2,3,-2.424598,36.830882,4940.0,5110.0,5510.0,5511.5,5544.5,5526.5,5565.5,5258.0,4971.0,4384.5,-9.153196,-15.161573
3,4,-2.412471,36.848489,5124.0,5292.0,5740.0,5635.0,5685.0,5709.0,5776.0,5392.0,4787.0,4020.0,-6.163539,-11.833104
4,5,-2.417861,36.858640,6442.0,6508.0,6848.0,6528.5,6801.0,6765.0,6668.0,6698.0,5869.0,5312.5,-6.095325,-14.596460


In [25]:
#Dropping the id Variable
y_train_kenya = (kenya_train_df["TARGET"]-1).to_numpy()
y_test_kenya = (kenya_test_df["TARGET"]-1).to_numpy()

y_train_vietnam = (vietnam_train_df["TARGET"]-1).to_numpy()
y_test_vietnam = (vietnam_train_df["TARGET"]-1).to_numpy()

y_train_spain = spain_train_df.to_numpy()
y_test_spain = spain_test_df.to_numpy()


#Dropping the id column
kenya_train_df = kenya_train_df.drop('ID', axis = 1)
kenya_test_df = kenya_test_df.drop('ID', axis = 1)

vietnam_train_df = vietnam_train_df.drop('ID', axis = 1)
vietnam_test_df = vietnam_test_df.drop('ID', axis = 1)

spain_train_df = spain_train_df.drop('ID', axis = 1)
spain_test_df = spain_test_df.drop('ID', axis = 1)



#converting to np arrays
x_train_kenya = kenya_train_df.to_numpy()
x_test_kenya = kenya_test_df.to_numpy()

x_train_vietnam = vietnam_train_df.to_numpy()
x_test_vietnam = vietnam_test_df.to_numpy()

x_train_spain = spain_train_df.to_numpy()
x_test_spain = spain_test_df.to_numpy()



KeyError: "['ID'] not found in axis"

In [26]:
#normalizing the data 
x_train_kenya = preprocessing.normalize(x_train_kenya)
x_test_kenya = preprocessing.normalize(x_test_kenya)

x_train_vietnam = preprocessing.normalize(x_train_vietnam)
x_test_vietnam = preprocessing.normalize(x_test_vietnam)

x_train_spain = preprocessing.normalize(x_train_spain)
x_test_spain= preprocessing.normalize(x_test_spain)


#Turning the data into torch torch tensors for gpu acceleration
x_train_kenya = torch.from_numpy(x_train_kenya).type(torch.float)
x_test_kenya = torch.from_numpy(x_test_kenya).type(torch.float)
y_train_kenya = torch.from_numpy(y_train_kenya).type(torch.float)
y_test_kenya = torch.from_numpy(y_test_kenya).type(torch.float)

x_train_vietnam = torch.from_numpy(x_train_vietnam).type(torch.float)
x_test_vietnam= torch.from_numpy(x_test_vietnam).type(torch.float)
y_train_vietnam = torch.from_numpy(y_train_vietnam).type(torch.float)
y_test_vietnam = torch.from_numpy(y_test_vietnam).type(torch.float)

x_train_spain = torch.from_numpy(x_train_spain).type(torch.float)
x_test_spain = torch.from_numpy(x_test_spain).type(torch.float)
y_train_spain = torch.from_numpy(y_train_spain).type(torch.float)
y_test_spain = torch.from_numpy(y_test_spain).type(torch.float)

In [27]:
#Defining Custom Dataset
class CountryDataset(Dataset):
    def __init__(self, x, y):
        self.features = x
        self.labels = y

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self. labels[idx]


In [31]:
#appending all data framees

# Convert DataFrames to tensors before concatenation
x_train_all = torch.cat((torch.tensor(kenya_train_df.values), 
                         torch.tensor(vietnam_train_df.values), 
                         torch.tensor(spain_train_df.values)))

x_test_all = torch.cat((torch.tensor(kenya_test_df.values), 
                        torch.tensor(vietnam_test_df.values), 
                        torch.tensor(spain_test_df.values)))

y_train_all = torch.cat((torch.tensor(y_train_kenya.values), 
                         torch.tensor(y_train_vietnam.values), 
                         torch.tensor(y_train_spain.values)))

y_test_all = torch.cat((torch.tensor(y_test_kenya.values), 
                        torch.tensor(y_test_vietnam.values), 
                        torch.tensor(y_test_spain.values)))


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 15 but got size 14 for tensor number 1 in the list.

In [6]:
batch_size = 32

#creating custom Datasets
train_dataset_kenya = CountryDataset(x_train_kenya, y_train_kenya)
test_dataset_kenya = CountryDataset(x_test_kenya, y_test_kenya)

train_dataset_vientam = CountryDataset(x_train_vietnam, y_train_vietnam)
test_dataset_vietnam = CountryDataset(x_test_vietnam, y_test_vietnam)

train_dataset = CountryDataset(x_train_spain, y_train_spain)
test_dataset = CountryDataset(x_test_spain, y_test_spain)

train_dataset = CountryDataset(x_train, y_train)
test_dataset = CountryDataset(x_test, y_test)



#creating custom DataLoaders
kenya_train_dl = DataLoader(train_dataset,batch_size = batch_size, shuffle = True)
kenya_test_dl = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [7]:
#Creating the Module
class First_Model(nn.Module):
    def __init__(self):
        super(First_Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(15, 128),  # Input size 15 features
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1),   # Output size is 1 for binary classification
            nn.Sigmoid()         # Use Sigmoid for binary classification
        )

    def forward(self, x):
        return self.net(x)


In [8]:
def training(model, loss_fn, train_loader, opt, epochs = 1000):

    running_losses = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0 
        for i, (datapoints, labels) in enumerate(tqdm(train_loader)):


            #forward proporgation
            outputs = model(datapoints)

            labels = labels.float().unsqueeze(1)
            loss = loss_fn(outputs, labels)

            #backpropogation
            opt.zero_grad()
            loss.backward()
            opt.step()


            running_losses.append(loss.item())
            epoch_loss+=loss.item()

            # if (i+1) %10 == 0:
            #     print(f"epoch number {epoch+1}, loss = {loss.item()}")
    print("Training Finished")
    return running_losses


In [9]:
def calculate_accuracy(model, test_loader):
    model.eval()  
    correct = 0
    total = 0

    with torch.no_grad():
        for datapoints, labels in test_loader:
            outputs = model(datapoints)

            #values between 0 and 1
            probabilities = torch.sigmoid(outputs)

            #binary classification
            predicted = (probabilities > 0.5).float()

            #adjusting shape
            labels = labels.unsqueeze(1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)


    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")
    return accuracy

In [10]:
model = First_Model()
loss_function  = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

running_losses = training(model = model, loss_fn = loss_function, train_loader = kenya_train_dl, opt = optimizer)

accuracy =  calculate_accuracy(model, kenya_test_dl)

100%|██████████| 32/32 [00:00<00:00, 1811.16it/s]


Training Finished
Test Accuracy: 88.37675350701403%
